<div align="right">June 2025 | &copy; JHU </div>

<img src="./figs/ep.logo.png" alt="JHU EP logo" width="200" align="right" />

# Retrieval-Augmented Generation
In this example, we will build and test a simple Retrieval-Augmented Generation with Large Language Models (RAG-LLM). It typically has two main components:
- Retriever identifies relevant context or documents from an external data source. It often uses embeddings to find and rank content based on similarity to the user's query. Popular retrievers include dense retrievers like Sentence Transformers, FAISS, or BM25-based systems. The retriever allows RAG-LLM to pull in precise and contextual information to aid the generative model in crafting accurate responses.
- Generators usually use a large language model, such as GPT or T5, which takes the retrieved context and the user's query as input and generates a coherent, contextually relevant answer. The generator relies on the retriever-provided context to produce detailed, accurate answers to specific questions.

The example below includes,
- The external data source is from the JSON file `squad-train-v2.0.json`, which is part of the SQuAD (Stanford Question Answering Dataset) v2.0, a popular dataset for training and evaluating machine learning models on reading comprehension and question-answering tasks. Stanford released this dataset and is widely used in natural language processing (NLP) for benchmarking question-answering systems (source: &#128214; **<a href="https://rajpurkar.github.io/SQuAD-explorer/" target="_blank">https://rajpurkar.github.io/SQuAD-explorer/</a>**)
- Retriever is a `Sentence Transformer` (source: &#129303; **<a href="https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2" target="_blank">https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2</a>**)
- Large Language Model (LLM) is a `Llama-3.2-1B-Instruct` model (source: &#9854;&#65039; **<a href="https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct" target="_blank">https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct</a>**)

Install the sentence transformer: `pip install sentence-transformers`&#8617;

Meta requires permission to download the model, which requires an `edu` email address. As in the previous lectures, follow the instructions to request permission.

Since the LLM we utilize in this example, `meta-llama/Llama-3.2-1B-Instruct`, can process the entire context, we will store only two key pieces of information: topics and contexts. At this time, we will not process `qas` section (question-answer section).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 72
from IPython.display import Markdown
import numpy as np
import pandas as pd
import os
import json
from sklearn.metrics.pairwise import cosine_similarity


MODEL_PATH= '/EP_models/'
os.environ['HF_HOME'] = MODEL_PATH  # before import transformers
os.environ['HF_DATASETS_OFFLINE']= '1'

In [ ]:
import torch
import transformers
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

# filter warnings
import warnings
transformers.logging.set_verbosity_error()
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f'PyTorch version= {torch.__version__}')
print(f'transformers version= {transformers.__version__}')
print(f'CUDA available= {torch.cuda.is_available()}')

In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
# explore SQuAD training data structure
with open('/EP_datasets/squad-train-v2.0.json', 'r') as f:
    squad_data = json.load(f)

n_titles = len(squad_data['data'])
print(f'There are {n_titles} titles/topics in SQuAD v2.0 training dataset.\n')

title_idx = 200
prg_idx = 2
current_title = squad_data['data'][title_idx]['title']
n_paragraphs = len(squad_data['data'][title_idx]['paragraphs'])
n_qas = len(squad_data['data'][title_idx]['paragraphs'][prg_idx]['qas'])
print(f"Title with index #{title_idx} is {squad_data['data'][title_idx]['title']}.")
print(f"Title {current_title} has {n_paragraphs} paragraphs. Paragraph with index {prg_idx} has {n_qas} question-answer pairs.")

# docs = [entry['context'] for entry in squad_data['data'][0]['paragraphs']]
# squad_data['data']

In [ ]:
# squad_data['data'][200]['paragraphs'][0]['qas']
squad_data['data'][200]['paragraphs'][0]['context']

In [ ]:
sqdata = {"topic": [], "context": []}

for i in range(n_titles):
    n_paragraphs = len(squad_data['data'][i]['paragraphs'])
    for j in range(n_paragraphs):
        sqdata["topic"].append(squad_data['data'][i]['title'])
        sqdata["context"].append(squad_data['data'][i]['paragraphs'][j]['context'])


df = pd.DataFrame(sqdata)

# sanity
print(f'Shape of df: {df.shape}')
df.head()

In [ ]:
class RAGLLM:
    def __init__(self, external_data):
        self.st_model = SentenceTransformer('all-MiniLM-L6-v2')  # sentence transformer is used for embeddings

        # external data
        self.external_data = external_data
        self.titles = self.external_data['topic'].unique()
        self.title_embeddings = self.st_model.encode(self.titles.tolist())  # embed each topic/title

        # LLM model
        self.generator = pipeline('text-generation', model='meta-llama/Llama-3.2-1B-Instruct',
                                  max_length=4096, device=Device)  # utilize GPU in this example

    def generate_answer(self, user_question):
        # Step 1: Find the best matching topic
        question_embedding = self.st_model.encode([user_question])
        title_similarities = cosine_similarity(question_embedding, self.title_embeddings)
        best_title_index = np.argmax(title_similarities)
        best_title = self.titles[best_title_index]
        print(f"The best topic: {best_title}\n")

        # Step 2: Filter contexts for the identified best topic
        topic_contexts = self.external_data[self.external_data['topic'] == best_title]['context'].tolist()

        # Step 3: Vectorize each context under the best topic and find the best matching context
        context_embeddings = self.st_model.encode(topic_contexts)
        context_similarities = cosine_similarity(question_embedding, context_embeddings)
        best_context_index = np.argmax(context_similarities)
        best_context = topic_contexts[best_context_index]
        print(f"The best paragraph: {best_context}\n")

        # Step 4: Generate the answer using LLM with retrieved context
        prompt = f"Context: {best_context}\n\nQuestion: {user_question}\nAnswer:"
        response = self.generator(prompt, max_length=300)
        answer_text = response[0]['generated_text']
        answer = answer_text.split("Answer: ")[-1]
        return answer


In [ ]:
query = "Who is the US president in 2012?"
ragllm = RAGLLM(df)
output = ragllm.generate_answer(query)
for part in output.split("\n\n"):
    print(part)

<div style="background-color: #F1E0D6; padding: 10px; border-radius: 5px;">
Analysis: The output is correct. The best topic from the external data source appears to be relevant, although the year in the best topic is 2004, while the question asks about 2012. The best paragraph mentions only President Bush, but the answer is Barack Obama which is correct.
</div>

In [ ]:
query = "What is the largest state in USA by area?"
ragllm = RAGLLM(df)
output = ragllm.generate_answer(query)
for part in output.split("\n\n"):
    print(part)

<div style="background-color: #F1E0D6; padding: 10px; border-radius: 5px;">
Analysis: The output is incorrect. The best topic from the external data source appears to be relevant. However, the best paragraph is quite off. It is also possible that the chosen topic wasn't the most relevant one. From the paragraph, we can see that the geography of the US basically discusses the geography of the US as a country, not its individual states. The information about the largest state in the US by area is possibly in the Alaska topic.
</div>

In [ ]:
query = "Which state is larger by area in the USA: Alaska or Texas?"
ragllm = RAGLLM(df)
output = ragllm.generate_answer(query)
for part in output.split("\n\n"):
    print(part)

<div style="background-color: #F1E0D6; padding: 10px; border-radius: 5px;">
Analysis: The output is correct. Both the best topic and the best paragraph from the external data source are on point.
</div>

In [ ]:
query = "What is the population of Florida?"
ragllm = RAGLLM(df)
output = ragllm.generate_answer(query)
for part in output.split("\n\n"):
    print(part)

<div style="background-color: #F1E0D6; padding: 10px; border-radius: 5px;">
Analysis: The output is correct. This question is quite general, as it does not mention a specific timeline. Therefore, the answer depends on how recent the external data source is. The last update to the SQuAD 2.0 dataset was in 2018. Hence, it makes sense that the output returned the population of Florida in 2015. Both the best topic and the best paragraph are relevant to the question.
</div>

In [ ]:
query = "Where is Mount Rushmore?"
ragllm = RAGLLM(df)
output = ragllm.generate_answer(query)
for part in output.split("\n\n"):
    print(part)

<div style="background-color: #F1E0D6; padding: 10px; border-radius: 5px;">
Analysis: The output is correct. However, the best topic and the best paragraph in the external dataset are not relevant.
</div>

## Analysis and Observations
The main purpose of the example above is to demonstrate a simple implementation of the RAG with an LLM. While this approach is useful, there are areas for improvement. Key points include:
1. External Dataset Update: The SQuAD 2.0 training dataset was last updated in 2018, while the Llama-3.2-1B-Instruct model was last updated in September 2024. As a result, the pre-trained LLM contains more recent information and data.
2. Dataset Limitations: The external dataset SQuAD 2.0 training dataset may have limitations that can cause the LLM to rely on pre-trained data instead of the external dataset for some outputs. This reflects the constraints of relying on solely static external sources.
3. Efficiency of Updates: In practice, updating external datasets is more efficient and optimized compared to updating the entire pre-trained data in an LLM. This efficiency contributes to the growing popularity and practicality of RAG-LLMs.
***
## Why RAG-LLM Is Better Than LLM Alone?
Using RAG-LLM is often better than using an LLM alone because RAG combines the strengths of both retrieval-based systems and generative language models. Here's why it can be advantageous:
1. Up-to-Date Knowledge: Accesses external, real-time data; LLMs rely on static, pre-training knowledge.
2. Improved Accuracy: Grounds responses in retrieved documents, reducing hallucinations.
3. Domain-Specific Flexibility: Tailored to specific domains by indexing relevant data.
4. Reduced Training Costs: Updates only the retrieval database, avoiding LLM retraining.
5. Transparency: Provides references or sources for answers, enhancing trust.
6. Cost Efficiency: Uses smaller models with retrieval for similar or better performance.

<div style="background-color: #F1E0D6; padding: 10px; border-radius: 5px;">
In the example below, we will illustrate Domain-Specific Flexibility. In the SQuAD dataset, under the 'Florida' topic, 'Florida' is mentioned with its meaning as 'flower' in Spanish. This information is quite specific and possibly unique. However, the pre-trained dataset of LLaMA may be more general. Therefore, while an LLM may not be able to answer the question correctly, a RAG-LLM will be able to do so in the Explanation part.
</div>

In [ ]:
class LLaMAQA:
    def __init__(self, model_name='meta-llama/Llama-3.2-1B-Instruct'):
        self.model_name = model_name
        print(f"Loading model '{model_name}'...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

    def ask_question(self, question, max_length=50, temperature=0.7, top_p=0.9, top_k=50):
        """
        Ask a question and get a response from the model.
        Parameters:
            question (str): The question to ask.
            max_length (int): Maximum length of the response.
            temperature (float): Sampling temperature for randomness.
            top_p (float): Nucleus sampling parameter.
            top_k (int): Top-k sampling parameter.
        Returns:
            str: The model's response.
        """
        inputs = self.tokenizer(question, return_tensors="pt")

        # generate the response
        outputs = self.model.generate(
            inputs["input_ids"],
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            do_sample=True
        )
        # decode and return the response
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
llama_qa = LLaMAQA()

query = "What does 'Florida' mean in Spanish?"
response = llama_qa.ask_question(query)
print("Q:", query)
print("A:", response)

In [ ]:
%%html
<style>
    table {margin-left: 0 !important;}
    p {font-family: verdana;}
    li {font-family: verdana;}
    div {font-size: 10pt;}
</style>
<!-- Display markdown tables left oriented in this notebook. -->

***
***